In [2]:
import cv2
import torchvision
import torch
from torchvision import transforms
import torchvision.transforms.v2

In [1]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

NameError: name 'torch' is not defined

In [4]:
mobile_net = torchvision.models.mobilenet_v2()
mobile_net.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=False),
    torch.nn.Linear(in_features=1280, out_features=1, bias=True),
    torch.nn.Sigmoid()
)
mobile_net.load_state_dict(torch.load("/home/leon/Downloads/epoch9.pth"))
mobile_net.eval()
mobile_net.to(device)
test_transforms = transforms.Compose([ 
    transforms.v2.Resize((128,128)), 
    transforms.v2.ToImage(), 
    transforms.v2.ToDtype(torch.float32, scale=True),
    transforms.v2.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225]),
])

In [10]:
vid = cv2.VideoCapture(0)
while(True): 
      
    # Capture the video frame 
    # by frame 
    ret, frame = vid.read() 
    
    if ret:
        # Display the resulting frame 
        cv2.imshow('frame', frame) 

        if not frame is None: 
            with torch.no_grad():
                print((mobile_net(test_transforms(frame).to(device).unsqueeze(dim=0)) > .5).item(),  end="\r")
        
    # the 'q' button is set as the 
    # quitting button you may use any 
    # desired button of your choice 
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
  
# After the loop release the cap object 
vid.release() 
# Destroy all the windows 
cv2.destroyAllWindows() 